# Wisdom Tree Assignment

We start by importing the libraries that we will need later on

In [6]:
import numpy as np
import pandas as pd
import os.path as pth

import matplotlib.pyplot as plt

from time import time
from WT_functions import IndexConstruction
from WT_functions import plot_single_contract

As first steps, we:
- Get the dataframes we will need to perform the analysis from the `.csv` files, namely `contracts_prices.csv` and `contracts_info.csv`.
- Transform date columns into `pd.datetime`.
- Sort the DataFrame by index first (trading date), then by maturity year and maturity month.
- Merge the contracts' information DataFrame with `df` in order to compute the metrics that are involving the `contract_size`. 

In [4]:
filename = 'contracts_prices.csv' 
filename_info = 'contracts_info.csv'

filepath = pth.join(pth.join('Other', 'WisdomTree'), filename)  # insert here the right path
filepath_info = pth.join(pth.join('Other', 'WisdomTree'), filename_info)  # insert here the right path

df = pd.read_csv(filepath).set_index('date')
df_info = pd.read_csv(filepath_info).set_index('contract_code')

# turn date columns into datetime dtype
df.index = pd.to_datetime(df.index, format='%d/%m/%Y')
df.loc[:, 'last_trade_date'] = pd.to_datetime(df['last_trade_date'], format='%d/%m/%Y')

df = df.reset_index().sort_values(['date', 'mat_year', 'mat_month'], ascending=[True, False, False]).set_index('date')
df = df.merge(df_info, left_on='contract_code', right_index=True)

In [ ]:
# we define the value of the contract by multiplying the current price by the size of the contract
df['value_USD'] = df.close.mul(df.contract_size)
df['oi_USD'] = df.oi.mul(df.value_USD).fillna(0)
df['volume_USD'] = df.volume.mul(df.value_USD).fillna(0)

lbls = ['contract_code', 'mat_month', 'mat_year']
# calculate the 22 days moving average (business days to get one full month moving average) and append as a column
volume_ma = df.groupby(lbls)['volume_USD'].rolling(window=22, min_periods=1).mean()
df = df.set_index(lbls, append=True).join(volume_ma, rsuffix='_1M_MA').reset_index().set_index('date')


In [5]:
df.head()

,contract_code,mat_month,mat_year,close,volume,oi,last_trade_date,contract_long_name,contract_short_name,underlying_name,contract_size,quote_currency,unit,value_USD,oi_USD,volume_USD,volume_USD_1M_MA
date,,,,,,,,,,,,,,,,,
2008-01-02,LA,2,2010,2637.0,NaN,1833.0,2010-02-15,LME Primary Aluminum,Aluminum,Aluminum,25,USD,Metric Tons,65925.0,1.208405e+08,0.0,0.0
2008-01-02,LA,1,2010,2632.0,NaN,11476.0,2010-01-18,LME Primary Aluminum,Aluminum,Aluminum,25,USD,Metric Tons,65800.0,7.551208e+08,0.0,0.0
2008-01-02,LA,12,2009,2627.0,180.0,36591.0,2009-12-14,LME Primary Aluminum,Aluminum,Aluminum,25,USD,Metric Tons,65675.0,2.403114e+09,11821500.0,11821500.0
2008-01-02,LA,11,2009,2621.0,NaN,3583.0,2009-11-16,LME Primary Aluminum,Aluminum,Aluminum,25,USD,Metric Tons,65525.0,2.347761e+08,0.0,0.0
2008-01-02,LA,10,2009,2615.0,1.0,2579.0,2009-10-19,LME Primary Aluminum,Aluminum,Aluminum,25,USD,Metric Tons,65375.0,1.686021e+08,65375.0,65375.0


todo weights proportional to roll yield

For each one of the commodity we have to find the *optimal future maturity* on the last day of each month. In order to to this, we first find the *optimal future maturity contract* for each day and then we will resapmple the dataframe found in order to sample the right contract at each month end.

With the **implied roll yield** defined as:

$$\text{Implied Roll Yield}_i = \bigg( \frac{\text{Price of contract}_{i-1}}{\text{Price of contract}_{i}} - 1 \bigg) \frac{1}{\text{# of months between the contracts}}$$

The **optimal future maturity** is defined, for each date, as the contract $i$ that satisfies:

$$\max_i \bigg( \text{Implied Roll Yield}_i \bigg)$$

Given that the it has an open interest of at least 100 USD/mln and 1-month average daily USD volume of at least 30 USD/mln.
